In [3]:
from selenium import webdriver
# Maybe you have to copy chromedriver.exe to project folder
import os
from os import path
import time
import re
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from bs4 import BeautifulSoup
import requests
import pandas as pd
from lxml import html

In [4]:
#filters = [type, nb lits, nb chambres,nb sdb]
def FilterPage(filters,driver):
  driver.maximize_window()
  

  url = 'https://www.airbnb.fr/s/Paris/homes?refinement_paths%5B%5D=%2Fhomes&search_type=filter_change&tab_id=home_tab&checkin=2021-01-08&checkout=2021-01-10&source=structured_search_input_header&min_bedrooms=0&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&min_beds=0'
  website = driver.get(url)
  time.sleep(5)
  more_filters = driver.find_elements_by_class_name("_w37zq5")
  if more_filters[2].is_displayed():
    driver.execute_script("arguments[0].click();", more_filters[2])
    time.sleep(1)

  checkbox = driver.find_elements_by_class_name("_py3ty1")
  
  if checkbox[filters[0]].is_displayed():
        driver.execute_script("arguments[0].click();", checkbox[filters[0]])
        time.sleep(1)

  save_type = driver.find_elements_by_class_name("_ajuxzjo")
  if save_type[0].is_displayed():
    driver.execute_script("arguments[0].click();", save_type[0])
    time.sleep(1)
  
  if more_filters[8].is_displayed():
    driver.execute_script("arguments[0].click();", more_filters[8])
    time.sleep(1)

  change_numbers = driver.find_elements_by_class_name("_7hhhl3")
  for i in range(filters[1]):
    # augmenter le nombre de lits
    if change_numbers[1].is_displayed():
        driver.execute_script("arguments[0].click();", change_numbers[1])
        time.sleep(1)
  for i in range(filters[2]):
    # augmenter le nombre de chambres
      if change_numbers[3].is_displayed():
          driver.execute_script("arguments[0].click();", change_numbers[3])
          time.sleep(1)

  for i in range(filters[3]):
    # augmenter le nombre de sdb
    if change_numbers[5].is_displayed():
            driver.execute_script("arguments[0].click();", change_numbers[5])
            time.sleep(1)
  save_morefilters = driver.find_elements_by_class_name("_m095vcq")
  if save_morefilters[0].is_displayed():
        driver.execute_script("arguments[0].click();", save_morefilters[0])
        time.sleep(1)

  


In [9]:
driver = webdriver.Chrome("C:\\Users\\Utilisateur\\Desktop\\chromedriver.exe") #mettre le bon path
FilterPage([1,1,1,1],driver)

In [ ]:
dataList = []
for n in range(1,2):
    for k in range(1,5):
      for j in range(1,5):
        for h in range(1,5):
            print('Données en cours de récupération : '+str(n)+ ' ; nb chambre='+str(k)+' ; nb cuisine='+str(j)+' ; nb sdb='+str(h))
            driver = driver = webdriver.Chrome("C:\\Users\\Utilisateur\\Desktop\\chromedriver.exe") #mettre le bon path
            filter =  [n,k,j,h]
            FilterPage(filter,driver)

            for i in range(0,15):

              url= driver.current_url
              page = requests.get(url, headers = {"Accept-Language": "en-US"})
              bs = BeautifulSoup(page.text, 'html.parser')


              page = requests.get(url, headers = {"Accept-Language": "en-US"})

              lodgings = bs.findAll('div', attrs={'itemprop': 'itemListElement'})
              prices=[]
              titles=[]
              for lodging in lodgings:
                  prices.append(lodging.findAll('span','_1p7iugi'))
                  titles.append(lodging.findAll('div','_167qordg'))


              Titles = []
              for p in range(len(titles)):
                  Titles.append(str(titles[p])[str(titles[p]).find('>')+1 : str(titles[p]).find('<', str(titles[p]).find('>'))])
              #print ("Found {} titles on the web page\n".format(len(Titles)))


              Prices = []
              for w in range(len(prices)):
                  if str(prices[w]).find('Prix réduit') >= 0:
                      Prices.append(str(prices[w])[str(prices[w]).find('</span>',str(prices[w]).find('Prix réduit'))+7:str(prices[w]).find('€',str(prices[w]).find('Prix réduit'))] + "€")
                  else:
                      Prices.append(str(prices[w])[str(prices[w]).find('</span>',str(prices[w]).find('Prix')) + 7 :str(prices[w]).find('€')] + "€")

              #print ("Found {} prices on the web page\n".format(len(Prices)))


              rooms = bs.findAll('div', '_kqh46o')
              #print ("Found {} rooms on the web page\n".format(len(rooms)))

              vals=[]
              for o in range(0,len(titles)):
                  vals.append({'title': Titles[o], 
                              'nb de voyageurs':rooms[2*o].contents[0], 
                              'nb de chambres':rooms[2*o].contents[2], 
                              'nb de lits':rooms[2*o].contents[4], 
                              'nb salles de bains':rooms[2*o].contents[6],
                              #'Note':quality[i].findAll('span','_krjbj')[0].contents[0],
                              #'nb de commentaires':quality[i].findAll('span','_krjbj')[1].contents[0],
                              'Prix':Prices[o]
                              })
              dataList.append(vals)



                #changer de page
              suiv = driver.find_elements_by_class_name("_za9j7e")
              if suiv[0].is_displayed():
                driver.execute_script("arguments[0].click();", suiv[0])
                time.sleep(1)
            driver.quit()
data=pd.DataFrame()
for i in range (len(dataList)):
    df=pd.DataFrame(dataList[i])
    data=pd.concat([data,df])
print(data)
df.drop_duplicates(keep = 'first', inplace=True)
data.to_csv(r'airbnb.csv', index = False)

In [13]:
data=pd.DataFrame()
for i in range (len(dataList)):
    df=pd.DataFrame(dataList[i])
    data=pd.concat([data,df])
print(data)
df.drop_duplicates(keep = 'first', inplace=True)
data.to_csv(r'airbnb.csv', index = False)

                                   title nb de voyageurs nb de chambres  \
0       Logement entier à Quartier Latin     5 voyageurs      1 chambre   
1           Logement entier à Courbevoie      1 voyageur      1 chambre   
2   Logement entier à 10e arrondissement     2 voyageurs      1 chambre   
3   Logement entier à 11e arrondissement     2 voyageurs      1 chambre   
4             Maison entière à Montreuil     5 voyageurs      1 chambre   
..                                   ...             ...            ...   
0            Logement entier à Vaugirard     6 voyageurs     3 chambres   
0     Maison entière à Herblay-sur-seine    16 voyageurs     5 chambres   
0   Logement entier à 16e arrondissement    11 voyageurs     5 chambres   
0     Maison entière à Herblay-sur-seine    16 voyageurs     5 chambres   
0     Maison entière à Herblay-sur-seine    16 voyageurs     5 chambres   

   nb de lits  nb salles de bains    Prix  
0      3 lits   1,5 salle de bain    196€  
1       1 l